In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import numpy as np
import logging
from src import models

from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

/home/local_arnab/miniconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('2.3.1', '4.43.3', '12.1')

In [3]:
from datasets import load_dataset

wiki = load_dataset("wikimedia/wikipedia", "20231101.en")

2024-08-21 12:46:51 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-08-21 12:46:51 numexpr.utils INFO     NumExpr defaulting to 8 threads.
2024-08-21 12:46:52 datasets INFO     PyTorch version 2.3.1 available.
2024-08-21 12:46:52 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-08-21 12:46:52 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/wikimedia/wikipedia HTTP/11" 200 164230
2024-08-21 12:46:52 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2024-08-21 12:46:52 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/wikimedia/wikipedia/wikimedia/wikipedia.py HTTP/11" 404 0
2024-08-21 12:46:52 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/wikimedia/wikipedia HTTP/11" 200 164230
2024-08-21 12:46:52 urllib3.connect

In [4]:
# print(len(wiki["train"]))
# wiki["train"][:5]

In [5]:
tiny = load_dataset("roneneldan/TinyStories")

2024-08-21 12:46:54 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/roneneldan/TinyStories HTTP/11" 200 1957
2024-08-21 12:46:54 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2024-08-21 12:46:54 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/roneneldan/TinyStories/roneneldan/TinyStories.py HTTP/11" 404 0
2024-08-21 12:46:54 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/roneneldan/TinyStories HTTP/11" 200 1957
2024-08-21 12:46:54 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-08-21 12:46:54 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/roneneldan/TinyStories/resolve/f54c09fd23315a6f9c86f9dc80f725de7d8f9c64/README.md HTTP/11" 200 0
2024-08-21 12:46:54 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-08-21 12:46

In [6]:
print(len(tiny["train"]))
tiny["train"][5055]

2119719


{'text': 'Once upon a time there was a dependable bear who loved having fun. He went on many adventures with his friends and each discovery was full of joy and surprise. One day, he found something he could not accept. His friends were scared, but the bear was brave. He thought carefully and tried to find a way to make things fun again. He worked hard and tried many things. Finally, he came up with a dependable way to solve the problem. All of the animals celebrated and had lots of fun! They thanked the bear for his hard work and accepted his solution. They all lived happily ever after. The end.'}

In [7]:
from src.models import ModelandTokenizer

# model_name = "openai-community/gpt2-xl"
# model_name = "openai-community/gpt2"
model_name = "EleutherAI/pythia-410m"

mt = ModelandTokenizer(
    model_key=model_name,
    # torch_dtype=torch.float16,
)

mt.n_embd, mt.n_layer

2024-08-21 12:46:56 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2024-08-21 12:46:56 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/EleutherAI/pythia-410m> | size: 875.114 MB | dtype: torch.float16 | device: cuda:0


(1024, 24)

In [8]:
mt.layer_name_format

'gpt_neox.layers.{}'

In [9]:
mt

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True

In [10]:
from dictionary_learning import ActivationBuffer
from dictionary_learning.training import trainSAE
from src.functional import get_module_nnsight

submodule = get_module_nnsight(mt, mt.layer_name_format.format(mt.n_layer//2))
activation_dim = mt.n_embd
dictionary_dim = 4096

# data_iter = iter(tiny["train"][:2100000]["text"])
# data_iter = iter(tiny["train"][:10000]["text"])
data_iter = iter(wiki["train"][:10000]["text"])


tiny_buffer = ActivationBuffer(
    data_iter,
    mt,
    submodule,
    d_submodule=activation_dim, # output dimension of the model component
    n_ctxs=300, # you can set this higher or lower dependong on your available memory
    device='cuda:0' # doesn't have to be the same device that you train your autoencoder on
) # buffer will return batches of tensors of dimension = submodule's output dimension

tiny_buffer.config

2024-08-21 12:46:56 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/sae/notebooks, stdin=None, shell=False, universal_newlines=False)
2024-08-21 12:46:56 git.cmd DEBUG    Popen(['git', 'version'], cwd=/home/local_arnab/Codes/sae/notebooks, stdin=None, shell=False, universal_newlines=False)
2024-08-21 12:46:56 wandb.docker.auth DEBUG    Trying paths: ['/home/local_arnab/.docker/config.json', '/home/local_arnab/.dockercfg']
2024-08-21 12:46:56 wandb.docker.auth DEBUG    No config file found


{'d_submodule': 1024,
 'io': 'out',
 'n_ctxs': 300,
 'ctx_len': 128,
 'refresh_batch_size': 512,
 'out_batch_size': 8192,
 'device': 'cuda:0'}

In [11]:
# steps = None
# for step, act in enumerate(tqdm(tiny_buffer, total=steps)):
#     print(step, act.shape)

In [12]:
from dictionary_learning.trainers.standard import StandardTrainer
from dictionary_learning.dictionary import AutoEncoder

# train the sparse autoencoder (SAE)
ae = trainSAE(
    data = tiny_buffer,
    trainer_configs= [{
        'trainer' : StandardTrainer,
        'dict_class' : AutoEncoder,
        'activation_dim' : activation_dim,
        'dict_size' : dictionary_dim,
        'lr' : 1e-3,
        'l1_penalty' : 1e-1,
        'warmup_steps' : 10000,
        'resample_steps' : None,
        'seed' : None,
        'wandb_name' : 'StandardTrainer',
        'lm_name': mt.name,
        'layer': submodule,
        'submodule_name': "residual",
    }],
    save_dir="trained_saes",
    save_steps=10,

    # use_wandb=True,
    # wandb_entity="dl-homeworks",
    # wandb_project="test_sae",
    # log_steps=5
)

0it [00:00, ?it/s]You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
57it [00:24,  2.30it/s]


In [13]:
from dictionary_learning.dictionary import AutoEncoder
from src.utils import env_utils

ae = AutoEncoder.from_pretrained(
    path = os.path.join(
        env_utils.DEFAULT_RESULTS_DIR,
        "trained_saes",
        mt.name.split("/")[-1],
        "TinyStories",
        "trainer_0/ae.pt"
    ),
    device='cuda:0'
)

In [14]:
ae

AutoEncoder(
  (encoder): Linear(in_features=1024, out_features=4096, bias=True)
  (decoder): Linear(in_features=4096, out_features=1024, bias=False)
)

In [15]:
ae.encoder.weight

Parameter containing:
tensor([[ 1.1105e-05,  2.0128e-02, -1.5454e-03,  ...,  5.5606e-03,
         -1.7302e-02,  1.3566e-02],
        [-7.5595e-03, -1.1574e-02, -2.3335e-02,  ...,  2.3730e-02,
          3.7004e-04,  1.7028e-02],
        [ 5.6901e-04,  2.5539e-02,  4.2656e-03,  ..., -6.8811e-03,
         -2.2053e-02,  1.1325e-02],
        ...,
        [ 6.6162e-04,  2.7501e-02, -1.2707e-02,  ..., -9.0291e-04,
          1.4508e-02,  2.9553e-02],
        [-1.1730e-02,  2.6030e-02, -2.9809e-03,  ..., -3.8242e-03,
         -2.0368e-02, -1.7750e-03],
        [ 2.0354e-02,  1.4571e-02,  1.3098e-02,  ..., -1.4957e-03,
          2.6515e-03, -9.9766e-03]], device='cuda:0', requires_grad=True)

In [1]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'